# Testing Data Modelling Techniques

This notebook is a complete mess. I'll make it clean later on (still testing some stuff).

Techniques to be tested:
- User2Vec
- Network Analysis (Tweets)
- Network Analysis (Users)
- Popular vs unpopular users identification
- Topic Modelling
- Text summarization

In [22]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import eland as ed
import networkx as nx

# uncomment this if database is not already open (and give ES a couple minutes to set up)
#!make database

es = Elasticsearch(['localhost'])

In [ ]:
# drop empty coluns
ed_df = ed.read_es('localhost', 'twitter')

df = ed_df[
    ['tweet_id', 'name', 'full_text_processed',
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
].to_pandas().fillna(np.nan)

## Network Analysis (Tweets)
There is possibly little to no information to be taken from this section, but I kept it anyway.

In [16]:
df_network = ed_df[
    ['tweet_id', 
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
].to_pandas().fillna(np.nan).rename(columns={'tweet_id': 'target'})

In [17]:
# remove the annoying scientific notation from id columns
pd.set_option('display.float_format', lambda x: '%.0f' % x)

def get_source(row):
    if row['is_quote_status']:
        return row['quoted_status_id_str']
    elif row['is_retweet']:
        return row['original_tweet_id_str']
    elif row['is_reply']:
        return row['in_reply_to_status_id_str']
    else:
        return np.nan

df_network['source'] = df_network.apply(get_source, axis=1)

In [20]:
df_network['weight'] = 1
df_network[['source', 'target', 'weight']]

,source,target,weight
1263804753286397952,1263801627288521984,1263804753286397952,1
1263820895312732160,1263812617430134785,1263820895312732160,1
1263820892871618560,1263762563566731264,1263820892871618560,1
1263820887897378816,1263788358343499782,1263820887897378816,1
1263820880800595968,1263820878783053056,1263820880800595968,1
...,...,...,...
1263850896217092096,1263848854383849477,1263850896217092096,1
1263850895160348672,1263848854383849477,1263850895160348672,1
1263850832103161856,1263847781094944768,1263850832103161856,1
1263850818689740800,1263848064592183298,1263850818689740800,1


In [32]:
# create a directed graph
G_dir = nx.from_pandas_edgelist(df_network[['source', 'target', 'weight']], create_using=nx.DiGraph())

# create an undirected graph
G = nx.from_pandas_edgelist(df_network[['source', 'target', 'weight']], create_using=nx.Graph())


In [37]:
print(f'Graph basic info: {G_dir.number_of_nodes()} nodes, {G_dir.number_of_edges()} edges.')

Graph basic info: 495233 nodes, 455471 edges.


In [38]:
# compare densities of the two graphs
print(f'Density measurement: \nUndirected: {nx.density(G)}\nDirected: {nx.density(G_dir)}')

Density measurement: 
Undirected: 3.71426127592167e-06
Directed: 1.857130637960835e-06


In [39]:
#out-degree
node_attrs=pd.DataFrame.from_dict(dict(G_dir.out_degree()), 'index').rename(columns={0:'out_degree'})

#in-degree
node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.in_degree()), 'index').rename(columns={0:'in_degree'}))

# weighted out-degree
#node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.out_degree(weight='weight')), 'index').rename(columns={0:'weighted_out_degree'}))

# weighted in-degree
#node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.in_degree(weight='weight')), 'index').rename(columns={0:'weighted_in_degree'}))

# add tweet ids and original info
#node_attrs=node_attrs.join(df.set_index('COMUNE')[["COD_REG","COD_CM", "COD_PRO", "PRO_COM"]])

In [48]:
node_attrs

,out_degree,in_degree
126,1,0
278816194440605696,1,0
401812009810800576,1,0
446602988501491712,1,0
521237811643445184,1,0
...,...,...
nan,1,0
nan,1,0
nan,1,0
nan,1,0


In [ ]:
node_attrs.out_degree.unique()

In [40]:
df_network[['source', 'target', 'weight']]

,source,target,weight
1263804753286397952,1263801627288521984,1263804753286397952,1
1263820895312732160,1263812617430134785,1263820895312732160,1
1263820892871618560,1263762563566731264,1263820892871618560,1
1263820887897378816,1263788358343499782,1263820887897378816,1
1263820880800595968,1263820878783053056,1263820880800595968,1
...,...,...,...
1263850896217092096,1263848854383849477,1263850896217092096,1
1263850895160348672,1263848854383849477,1263850895160348672,1
1263850832103161856,1263847781094944768,1263850832103161856,1
1263850818689740800,1263848064592183298,1263850818689740800,1


## Network Analysis (users)

In [66]:
df_network = ed_df[
    ['tweet_id', 'name',
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
].to_pandas().fillna(np.nan)

In [61]:
cols = ed_df.columns
cols[cols.str.contains('name')]

#ed_df[[]]

Index(['derived.locations.full_name', 'entities.user_mentions.name',
       'entities.user_mentions.screen_name', 'name', 'screen_name',
       'users_derived_full_name'],
      dtype='object')

## User2Vec
Based on: https://ieeexplore.ieee.org/document/8875952/

How it works:
- Run doc2vec
- Average vector representations for each user


In [70]:
from gensim.models.doc2vec import Doc2Vec

In [ ]:
df_